In [1]:
import numpy
from scipy.linalg import solve

In [3]:
def constructMatrix(nx, ny, sigma):
    """Generate implicit matrix for 2D heat equation with Dirichlet in bottom and right and Neumann in top and left
    Assumes dx = dy
    
    Parameters:
    ----------
    nx: int
        number of discretization points in x
    
    ny: int 
        number of discretization points in y
    
    sigma: float
        alpha*dt/dx
        
    Returns: 
    --------
    A: 2D array of floats 
        Matrix
    """
    
    A = numpy.zeros(((nx-2)*(ny-2),(nx-2)*(ny-2)))
    
    row_number = 0 #row counter
    for j in range(1, ny-1):
        for i in range(1,nx-1):
            
            #Corners
            if i==1 and j==1: #Bottom left corner (Dirichlet down and left)
                A[row_number, row_number] = 1/sigma+4 #Set diagonal
                A[row_number, row_number+1] = -1 #fetch i+1
                A[row_number, row_number+nx-2] = -1 #fetch j+1
                
            elif i==nx-2 and j==1: #Bottom right corner (Dirichlet down, Neumann right)
                A[row_number, row_number] = 1/sigma+3 #Set diagonal
                A[row_number, row_number-1] = -1  #Fetch i-1
                A[row_number, row_number+nx-2] = -1 #fetch j+1
                
            elif i==1 and j==ny-2: # Top left corner (Neumann up, Dirichlet left)
                A[row_number,row_number] = 1/sigma+3   # Set diagonal
                A[row_number,row_number+1] = -1        # fetch i+1
                A[row_number,row_number-(nx-2)] = -1   # fetch j-1
                
            elif i==nx-2 and j==ny-2: # Top right corner (Neumann up and right)
                A[row_number,row_number] = 1/sigma+2   # Set diagonal
                A[row_number,row_number-1] = -1        # Fetch i-1
                A[row_number,row_number-(nx-2)] = -1   # fetch j-1
              
            # Sides
            elif i==1: # Left boundary (Dirichlet)
                A[row_number,row_number] = 1/sigma+4 # Set diagonal
                A[row_number,row_number+1] = -1      # fetch i+1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                A[row_number,row_number-(nx-2)] = -1 # fetch j-1
            
            elif i==nx-2: # Right boundary (Neumann)
                A[row_number,row_number] = 1/sigma+3 # Set diagonal
                A[row_number,row_number-1] = -1      # Fetch i-1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                A[row_number,row_number-(nx-2)] = -1 # fetch j-1
                
            elif j==1: # Bottom boundary (Dirichlet)
                A[row_number,row_number] = 1/sigma+4 # Set diagonal
                A[row_number,row_number+1] = -1      # fetch i+1
                A[row_number,row_number-1] = -1      # fetch i-1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                
            elif j==ny-2: # Top boundary (Neumann)
                A[row_number,row_number] = 1/sigma+3 # Set diagonal
                A[row_number,row_number+1] = -1      # fetch i+1
                A[row_number,row_number-1] = -1      # fetch i-1
                A[row_number,row_number-(nx-2)] = -1 # fetch j-1
                
            # Interior points
            else:
                A[row_number,row_number] = 1/sigma+4 # Set diagonal
                A[row_number,row_number+1] = -1      # fetch i+1
                A[row_number,row_number-1] = -1      # fetch i-1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                A[row_number,row_number-(nx-2)] = -1 # fetch j-1
                
            row_number += 1 # Jump to next row of the matrix!
    
    return A